### generate_synthetic_principles_for_agents.ipynb
This notebook generates an agents synthetic principles for use in HCVA
- Takes an individual agents value system
    - finds the decision they would take, and assigns the agent an ethical principle that fits with their view, or is the opposite of their view according to some probability $p$

In [2]:
import pandas as pd
import numpy as np

#####
# 1.0 = Utilitarian, 1.4 = Transition point, 2.1 = HCVA, 10.0 = Egalitarian
###
relevent_consensuses =  [1.0, 1.4, 2.1, 10.0]
#relevent_consensuses = [round(x * 0.1, 1) for x in range(10, 101)]
relevant_columns = ['p','Rel-Nonrel', 'Nonrel-Rel', 'Rel_div_p', 'Nonrel_div_p', 'Egal-Util', 'Util-Egal']

actions_filename = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/results/02-01-2025-actions-a-2.5-p-5.0.csv"
preference_filename = '/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/results/02-01-2025-preferences-a-2.5-p-5.0.csv'

final_df_savename = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/02-01-2025-ess-relevant-consensus.csv"

pref_df = pd.read_csv(preference_filename)
act_df = pd.read_csv(actions_filename)
cons_df = pd.merge(pref_df, act_df, on='p')
cons_df['p'] = cons_df['p'].round(1)

In [3]:
# for each row, find the corresponding Util-Egal and Egal-Util values (where P=1.0 has Util-Egal=1.0 and Egal-Util=0.0, and P=10.0 has Util-Egal=0.0 and Egal-Util=1.0)
for index, row in cons_df.iterrows():
    cons_df.at[index, 'Egal-Util'] = (row['p'] - 1) / 9
    cons_df.at[index, 'Util-Egal'] = 1 - cons_df.at[index, 'Egal-Util']

# As of 14/11, using processed_data_one_action_with_factor_2.5_5.0 P values 1.0-1.7 are for the scheme, 1.8 - 10.0 are against
against_scheme = list(np.arange(1.8, 10.0, 0.1))
for_scheme = list(np.arange(1.0, 1.7, 0.1))
against_scheme = [round(num, 1) for num in against_scheme]
for_scheme = [round(num, 1) for num in for_scheme]

final_df = pd.DataFrame()
for consensus in relevent_consensuses:
    filtered_df = cons_df[cons_df['p'] == consensus][relevant_columns]
    filtered_df['decision'] = (filtered_df['Rel_div_p'] * filtered_df['Rel-Nonrel']) + (filtered_df['Nonrel_div_p'] * filtered_df['Nonrel-Rel'])
    final_df = pd.concat([final_df, filtered_df], ignore_index=True)
final_df.to_csv(final_df_savename)
final_df

,p,Rel-Nonrel,Nonrel-Rel,Rel_div_p,Nonrel_div_p,Egal-Util,Util-Egal,decision
0,1.0,0.346411,0.653589,0.010277,0.144976,0.000000,1.000000,0.098315
1,1.4,0.345776,0.654224,-0.089210,0.077451,0.044444,0.955556,0.019824
2,2.1,0.352679,0.647321,-0.113565,0.042727,0.122222,0.877778,-0.012394
3,10.0,0.371893,0.628107,-0.042340,0.007190,1.000000,0.000000,-0.011230


In [4]:
# Read in data
agent_csv_file = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/processed_data_one_action_ess.csv_with_factor_2.5_5.0.csv"
df = pd.read_csv(agent_csv_file)
# sanity check
print(df.head())

   Unnamed: 0 country   rel  nonrel  a_div_rel  a_div_nonrel  Rel-Nonrel  \
0           0      AT   642    1216   0.238837      0.561952    0.731701   
1           1      BE   621    1116  -0.319377     -0.271804    0.713731   
2           2      CH   767     666   0.754020      1.000000    0.447139   
3           3      CZ   577    1483   0.106875      0.144976    0.829854   
4           4      DE  1264    1488   0.010549      0.340502    0.561047   

   Nonrel-Rel  
0    0.268299  
1    0.286269  
2    0.552861  
3    0.170146  
4    0.438953  


In [5]:
def binary_principles(df):
    df['decision'] = np.nan  # Initialize the 'decision' column

    for index, row in df.iterrows():
        df.at[index, 'decision'] = (row['rel-nonrel'] * row['a_div_rel']) + (row['nonrel-rel'] * row['a_div_nonrel'])
        if df.at[index, 'decision'] < 0:
            df.at[index, 'egal'] = 1
            df.at[index, 'util'] = 0
        else:
            df.at[index, 'egal'] = 0
            df.at[index, 'util'] = 1
    return df

def range_principles(df):
    df['decision'] = np.nan  # Initialize the 'decision' column
    for index, row in df.iterrows():
        # note: against_scheme = 1.8 to 3.1
                # for_scheme = 1.0 to 1.7 and 3.2 to 10.0
        total_interviewees = row['rel'] + row['nonrel']

        df.at[index, 'decision'] = (row['Rel-Nonrel'] * row['a_div_rel']) + (row['Nonrel-Rel'] * row['a_div_nonrel'])
        if df.at[index, 'decision'] < 0:
            factor = np.random.choice(against_scheme)
            egal = (factor / 10) * total_interviewees
            util = total_interviewees - egal
            egal = round(egal, 0)
            util = round(util, 0)
            # Factor is 1.0 then [egal, util] = [0, 1]
            # Factor is 10.0 then [egal, util] = [1, 0]
            # Factor is 1.8 then [egal, util] = [0.18, 0.82]

            df.at[index, 'egal'] = egal
            df.at[index, 'util'] = util
        else:
            factor = np.random.choice(for_scheme)
            egal = (factor / 10) * total_interviewees
            util = total_interviewees - egal
            egal = round(egal, 0)
            util = round(util, 0)
            df.at[index, 'egal'] = egal
            df.at[index, 'util'] = util
    return df


In [6]:
df = range_principles(df)
df

,Unnamed: 0,country,rel,nonrel,a_div_rel,a_div_nonrel,Rel-Nonrel,Nonrel-Rel,decision,egal,util
0,0,AT,642,1216,0.238837,0.561952,0.731701,0.268299,0.325528,297.0,1561.0
1,1,BE,621,1116,-0.319377,-0.271804,0.713731,0.286269,-0.305759,1529.0,208.0
2,2,CH,767,666,0.754020,1.000000,0.447139,0.552861,0.890013,229.0,1204.0
3,3,CZ,577,1483,0.106875,0.144976,0.829854,0.170146,0.113357,330.0,1730.0
4,4,DE,1264,1488,0.010549,0.340502,0.561047,0.438953,0.155383,303.0,2449.0
5,5,EE,802,1118,0.361596,0.247466,0.623437,0.376563,0.318619,192.0,1728.0
6,6,ES,567,1143,0.073486,0.281423,0.752632,0.247368,0.124923,274.0,1436.0
7,7,FI,739,1099,-0.493911,-0.028814,0.646899,0.353101,-0.329685,1562.0,276.0
8,8,FR,1063,899,0.105049,0.561735,0.437309,0.562691,0.362022,196.0,1766.0
9,9,GB,541,1287,0.040049,0.252525,0.806072,0.193928,0.081254,238.0,1590.0


In [7]:
# Save the principles a file
principles_df = df[['country', 'egal', 'util']]
principles_df.rename(columns={'egal': 'rel', 'util': 'nonrel'}, inplace=True)

## principles_df will give you just the principles, df will give you the
##  principles and all other data for each agent
# principles_df.to_csv("/home/ia23938/Documents/GitHub/ValueSystemsAggregation/process_data/14-11-processed_data_with_principles_ess.csv", index=False)
# df.to_csv("/home/ia23938/Documents/GitHub/ValueSystemsAggregation/process_data/14-11-processed_data_with_principles_ess.csv", index=False)


/tmp/ipykernel_71834/376870079.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  principles_df.rename(columns={'egal': 'rel', 'util': 'nonrel'}, inplace=True)
